In [1]:
import polars as pl  
import pandas as pd
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime, timedelta
import random
import onnx

In [3]:
def load_data(num_records=100):
    stock_symbols = ["AAPL", "GOOG", "MSFT", "AMZN", "TSLA", "NFLX"]
    base_date = datetime(2024, 11, 19, 12, 0, 0)
    
    records = []
    for _ in range(num_records):
        stock_symbol = random.choice(stock_symbols)
        current_price = round(random.uniform(100, 3000), 2)
        change = round(random.uniform(-10, 10), 2)
        previous_close_price = current_price - change
        percent_change = round((change / previous_close_price) * 100, 2) if previous_close_price else 0
        high_price_of_the_day = round(current_price + random.uniform(0, 20), 2)
        low_price_of_the_day = round(current_price - random.uniform(0, 20), 2)
        open_price_of_the_day = round(previous_close_price + random.uniform(-5, 5), 2)
        record_datetime = base_date + timedelta(minutes=random.randint(0, 1440))
        
        records.append({
            "stock_symbol": stock_symbol,
            "current_price": current_price,
            "change": change,
            "percent_change": percent_change,
            "high_price_of_the_day": high_price_of_the_day,
            "low_price_of_the_day": low_price_of_the_day,
            "open_price_of_the_day": open_price_of_the_day,
            "previous_close_price": previous_close_price,
            "Datetime": record_datetime.strftime("%Y-%m-%d %H:%M:%S"),
            "Date": record_datetime.strftime("%Y-%m-%d"),
            "Day": record_datetime.day,
            "Month": record_datetime.month,
            "Year": record_datetime.year,
            "Hour": record_datetime.hour,
            "Min": record_datetime.minute,
            "Sec": record_datetime.second,
        })

    return pl.DataFrame(records)
silver_stock_prices_fact = load_data()

In [ ]:
def preprocess_data(data):
        df = data.with_columns([
            (pl.col("Datetime").str.strptime(pl.Datetime)).alias("Datetime")
        ])
        numeric_cols = ["current_price", "change", "percent_change", "high_price_of_the_day",
                        "low_price_of_the_day", "open_price_of_the_day", "previous_close_price"]
        df = df.select(numeric_cols)
        # Scaling
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df.to_pandas())
        return df_scaled

data_scaled = preprocess_data(silver_stock_prices_fact)

In [5]:
def create_sequences(data, sequence_length=60):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length, 0])  # Predicting the current_price
    return np.array(X), np.array(y)

X, y = create_sequences(data_scaled)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # Take the last output of LSTM
        lstm_out = lstm_out[:, -1, :]
        return self.fc(lstm_out)

def build_model( input_size, hidden_size=64, num_layers=2, output_size=1):
    model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
    return model

model= build_model(input_size=X_train.shape[2]) 

In [10]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=32, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1))
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor.unsqueeze(1))
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")
    return model

model_ = train_model(model, X_train, y_train, X_val, y_val)

Epoch 1/10, Loss: 0.28703856468200684, Val Loss: 0.2916637063026428
Epoch 2/10, Loss: 0.2560417652130127, Val Loss: 0.25490081310272217
Epoch 3/10, Loss: 0.22717790305614471, Val Loss: 0.21970653533935547
Epoch 4/10, Loss: 0.20021559298038483, Val Loss: 0.1858130693435669
Epoch 5/10, Loss: 0.17506296932697296, Val Loss: 0.15315622091293335
Epoch 6/10, Loss: 0.15186472237110138, Val Loss: 0.12200917303562164
Epoch 7/10, Loss: 0.13113407790660858, Val Loss: 0.09315484762191772
Epoch 8/10, Loss: 0.11392109841108322, Val Loss: 0.06816042959690094
Epoch 9/10, Loss: 0.10203973948955536, Val Loss: 0.049699608236551285
Epoch 10/10, Loss: 0.0981411337852478, Val Loss: 0.041005317121744156


In [13]:
def save_model_to_onnx(model_, filepath="model.onnx", input_size=7, sequence_length=60):
        dummy_input = torch.randn(1, sequence_length, input_size).to(device)
        torch.onnx.export(model_, dummy_input, filepath, export_params=True, opset_version=11)
        print(f"Model saved as {filepath}")
    
save_model_to_onnx(model_)

Model saved as model.onnx


C:\Users\NolanM\Desktop\Seminar\venv\lib\site-packages\torch\onnx\symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [ ]:
class StockPricePredictionModel:
    def __init__(self):
        self.scaler = None
        self.model = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def load_data(self, num_records=100):
        stock_symbols = ["AAPL", "GOOG", "MSFT", "AMZN", "TSLA", "NFLX"]
        base_date = datetime(2024, 11, 19, 12, 0, 0)
        
        records = []
        for _ in range(num_records):
            stock_symbol = random.choice(stock_symbols)
            current_price = round(random.uniform(100, 3000), 2)
            change = round(random.uniform(-10, 10), 2)
            previous_close_price = current_price - change
            percent_change = round((change / previous_close_price) * 100, 2) if previous_close_price else 0
            high_price_of_the_day = round(current_price + random.uniform(0, 20), 2)
            low_price_of_the_day = round(current_price - random.uniform(0, 20), 2)
            open_price_of_the_day = round(previous_close_price + random.uniform(-5, 5), 2)
            record_datetime = base_date + timedelta(minutes=random.randint(0, 1440))
            
            records.append({
                "stock_symbol": stock_symbol,
                "current_price": current_price,
                "change": change,
                "percent_change": percent_change,
                "high_price_of_the_day": high_price_of_the_day,
                "low_price_of_the_day": low_price_of_the_day,
                "open_price_of_the_day": open_price_of_the_day,
                "previous_close_price": previous_close_price,
                "Datetime": record_datetime.strftime("%Y-%m-%d %H:%M:%S"),
                "Date": record_datetime.strftime("%Y-%m-%d"),
                "Day": record_datetime.day,
                "Month": record_datetime.month,
                "Year": record_datetime.year,
                "Hour": record_datetime.hour,
                "Min": record_datetime.minute,
                "Sec": record_datetime.second,
            })
        self.silver_stock_prices_fact = pl.DataFrame(records)
        return self.silver_stock_prices_fact

    def preprocess_data(self):
        df = self.silver_stock_prices_fact.with_columns([
            (pl.col("Datetime").str.strptime(pl.Datetime)).alias("Datetime")
        ])
        numeric_cols = ["current_price", "change", "percent_change", "high_price_of_the_day",
                        "low_price_of_the_day", "open_price_of_the_day", "previous_close_price"]
        df = df.select(numeric_cols)
        # Scaling
        self.scaler = MinMaxScaler()
        df_scaled = self.scaler.fit_transform(df.to_pandas())
        return df_scaled

    def create_sequences(self, data, sequence_length=60):
        X, y = [], []
        for i in range(len(data) - sequence_length):
            X.append(data[i:i + sequence_length])
            y.append(data[i + sequence_length, 0])  # Predicting the current_price
        return np.array(X), np.array(y)

    class LSTMModel(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, output_size):
            super().__init__()
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
            self.fc = nn.Linear(hidden_size, output_size)

        def forward(self, x):
            lstm_out, _ = self.lstm(x)
            # Take the last output of LSTM
            lstm_out = lstm_out[:, -1, :]
            return self.fc(lstm_out)

    def build_model(self, input_size, hidden_size=64, num_layers=2, output_size=1):
        self.model = self.LSTMModel(input_size, hidden_size, num_layers, output_size).to(self.device)

    def train_model(self, X_train, y_train, X_val, y_val, epochs=10, batch_size=32, learning_rate=0.001):
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(self.device)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(self.device)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(self.device)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(self.device)

        train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            self.model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y.unsqueeze(1))
                loss.backward()
                optimizer.step()

            self.model.eval()
            with torch.no_grad():
                val_outputs = self.model(X_val_tensor)
                val_loss = criterion(val_outputs, y_val_tensor.unsqueeze(1))
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    def save_model_to_onnx(self, filepath="model.onnx", input_size=7, sequence_length=60):
        dummy_input = torch.randn(1, sequence_length, input_size).to(self.device)
        torch.onnx.export(self.model, dummy_input, filepath, export_params=True, opset_version=11)
        print(f"Model saved as {filepath}")

    def process(self):
        self.load_data()
        data_scaled = self.preprocess_data()

        X, y = self.create_sequences(data_scaled)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        self.build_model(input_size=X_train.shape[2])
        self.train_model(X_train, y_train, X_val, y_val)
        self.save_model_to_onnx()

if __name__ == "__main__":
    stock_model = StockPricePredictionModel()
    stock_model.process()

Epoch 1/10, Loss: 0.36935168504714966, Val Loss: 0.3595671057701111
Epoch 2/10, Loss: 0.33783218264579773, Val Loss: 0.32574552297592163
Epoch 3/10, Loss: 0.30832916498184204, Val Loss: 0.2930193543434143
Epoch 4/10, Loss: 0.28015708923339844, Val Loss: 0.26085108518600464
Epoch 5/10, Loss: 0.2529056668281555, Val Loss: 0.22890505194664001
Epoch 6/10, Loss: 0.22636990249156952, Val Loss: 0.19697800278663635
Epoch 7/10, Loss: 0.20050835609436035, Val Loss: 0.1650407761335373
Epoch 8/10, Loss: 0.17550356686115265, Val Loss: 0.13335151970386505
Epoch 9/10, Loss: 0.15189111232757568, Val Loss: 0.10265946388244629
Epoch 10/10, Loss: 0.13077639043331146, Val Loss: 0.07456889003515244
Model saved as model.onnx


C:\Users\NolanM\Desktop\Seminar\venv\lib\site-packages\torch\onnx\symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
